In [1]:
import extFunc
import numpy as np
import os
from importlib import import_module
import copy 
import pandas as pd
import sys
import cobra

In [2]:
#Load Models if not already in RAM

#nameList = ['iJN1462_GLC_UR10_9.xml' , 'iJN1462_GLC_UR6_3.xml', 'iJN1462_GLN_UR5_1.xml', 'iJN1462_GLC_UR7_3.xml' , 'iJN1462_OCT_UR3_4.xml']

nameList = ['iJN1462_GLC_UR10_9.xml' , 'iJN1462_GLC_UR6_3.xml']

if 'modelDict' not in locals():
    modelDict = extFunc.ImportFunction(nameList)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
class ModelComparison():
    '''
    This Class can be used to compare multiple modelversions 
    and to get an overview of the C-Flux routes and 
    the corresponding fva
    input  = list of models
    '''

    def __init__(self, modelDict: "modelDict", **kwargs):
        self.modelDict = modelDict
        self.solutionDict = {}
        self.summaryDict = {}
        self.dataFrameDict = {}
        self.generate()

    # collect the objects of interest
    def generate(self):
        for model in self.modelDict.keys():
            self.solutionDict[model] = self.modelDict[model].optimize()
            self.summaryDict[model] = self.modelDict[model].summary(
                solution=self.solutionDict[model], fva=.99)
            self.dataFrameDict[model] = {
                "uptake":   pd.read_html(self.summaryDict[model].to_html())[0].sort_values('C-Flux', ascending=False),
                "secretion":   pd.read_html(self.summaryDict[model].to_html())[1].sort_values('C-Flux', ascending=False)
            }

    def mySummary(self):
        dfList, keyList = [], []
        '''
        groups          = ["uptake" , "secretion", "objective"]
        quantities      = ["glc" , "oct", "bio"]
        ids             = ["EX_glc__D_e" ,"EX_octa_e", "BIOMASS_KT2440_WT3" ]

        for model in self.modelDict.keys():
            df2 = pd.DataFrame(                
                list(comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes[["EX_glc__D_e" ,"EX_octa_e", "BIOMASS_KT2440_WT3" ]]), 
                quantities,groups
            )
            key
        dfList.append(df2) , keyList.append(model)
        '''
    
    
    

In [4]:
if 'clomp' not in locals():
    comp = ModelComparison(modelDict=modelDict)

In [21]:
#originale
'''
def mySummaryFunc(comparison,nrRea=3):
    dfList, keyList = [], []
    
    for model in comparison.modelDict.keys(): 
        theMatrix   = [ comp.dataFrameDict[model]['uptake'].iloc[0:3],
                        comp.dataFrameDict[model]['secretion'].iloc[0:3] ] 
     
        modelFrame=pd.concat(
            [pd.concat(theMatrix,
            keys=['Uptake', 'Secretion'], axis=1)],
            keys=[model]
            ) 
        dfList.append(modelFrame)
        keyList.append(model)

    return pd.concat(dfList,axis=0)
'''

def mySummaryFunc(comparison,nrRea=3):
    dfList, keyList = [], []
    
    for model in comparison.modelDict.keys(): 
        theMatrix   = [ comp.dataFrameDict[model]['uptake'].iloc[0:3].reset_index(drop=True),
                        comp.dataFrameDict[model]['secretion'].iloc[0:3].reset_index(drop=True) ] 
     
        modelFrame=pd.concat(
            [pd.concat(theMatrix,
            keys=['Uptake', 'Secretion'], axis=1)],
            keys=[model]
            ) 
        dfList.append(modelFrame)
        keyList.append(model)

    return pd.concat(dfList,axis=0)

    '''    
            df2 = pd.DataFrame(
                    [list(comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes[["EX_glc__D_e", "EX_octa_e", "BIOMASS_KT2440_WT3"]]),
                    list(comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes[["EX_glc__D_e", "EX_octa_e", "BIOMASS_KT2440_WT3"]]),
                    list(comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes[["EX_glc__D_e", "EX_octa_e", "BIOMASS_KT2440_WT3"]])
                    ],quantities, groups )

    return df
    '''

a = mySummaryFunc(comp)
a


Uptake                           \
                      Metabolite      Reaction       Flux   
iJN1462_GLC_UR10_9 0   glc__D[e]   EX_glc__D_e  10.900000   
                   1      ca2[e]      EX_ca2_e   0.002422   
                   2      na1[e]      EX_na1_e   0.002019   
iJN1462_GLC_UR6_3  0   glc__D[e]   EX_glc__D_e   6.300000   
                   1  pqqA_kt[c]  SK_pqqA_kt_c   0.000137   
                   2      na1[e]      EX_na1_e   0.002168   

                                                                 Secretion  \
                                       Range C-Number   C-Flux  Metabolite   
iJN1462_GLC_UR10_9 0           [10.84; 10.9]        6  100.00%     glcn[e]   
                   1    [0.002398; 0.002422]        0    0.00%  2dhglcn[e]   
                   2    [0.001998; 0.002019]        0    0.00%      co2[e]   
iJN1462_GLC_UR6_3  0            [6.237; 6.3]        6   99.99%      co2[e]   
                   1  [0.0001359; 0.0001369]       14    0.01%    5drib[c]   
                   2    [0.002147; 0.002168]        0    0.00%     amob[c]   

                                                                         \
                          Reaction       Flux                     Range   
iJN1462_GLC_UR10_9 0     EX_glcn_e  -2.800000            [-2.861; -2.8]   
                   1  EX_2dhglcn_e  -2.600000            [-2.664; -2.6]   
                   2      EX_co2_e  -9.920000          [-10.15; -9.667]   
iJN1462_GLC_UR6_3  0      EX_co2_e -13.010000          [-13.26; -12.74]   
                   1    DM_5drib_c  -0.000412    [-0.01385; -0.0004077]   
                   2     DM_amob_c  -0.000137  [-0.0001373; -0.0001359]   

                                       
                     C-Number  C-Flux  
iJN1462_GLC_UR10_9 0        6  39.69%  
                   1        6  36.86%  
                   2        1  23.44%  
iJN1462_GLC_UR6_3  0        1  99.97%  
                   1        5   0.02%  
                   2       15   0.02%

Trash Code

'tets'
#comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes.get_by_id('EX_glc__D_e')
#comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes.get_by_id('EX_octa_e')
#comp.solutionDict[['iJN1462_GLC_UR10_9']]
#b = comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes['EX_glc__D_e']
#comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes.get_by_id('EX_octa_e')
#a = comp.solutionDict['iJN1462_GLC_UR10_9']
#comp.modelDict['iJN1462_GLC_UR10_9'].summary()
'''
def createOutputWrapper(modelList):

    pass
    for model in modelList:
        tableForDisppay.join(modelList)      

    return tableForDisppay

def createOutput(model):
    sol = model.optimize()
    sum = model.summary(solution = sol,fva=.99)
    
    #biomass = solution.reac ('BIOMASS_KT2440_WT3') 
    print(biomass)
'''
    

#createOutput(modelList)       
#das gesamte Paket als 
#df = iJN1462_OCT_UR3_4.summary().to_frame()

#iJN1462_OCT_UR3_4.tolerance = 1e-1
#iJN1462_OCT_UR3_4.summary()
#dhtml =de.to_html()
#df = pd.read_html(dhtml)
#pd.DataFrame(df[1]).sort_values('C-Flux',ascending=False)
#df[1]
#
#de
#iJN1462_OCT_UR3_4.summary()
#iJN1462_OCT_UR3_4._tolerance = 2
#solutionSummaryObject = iJN1462_OCT_UR3_4.summary(fva=.99)


#pd.DataFrame(pd.read_html(dhtml)[1]).sort_values('C-Flux',ascending=False)


#iJN1462_OCT_UR3_4.reactions.get_by_id('biomass')
#createOutput(iJN1462_GLC_UR10_9)
